In [15]:
import csv
import sys
import os
import random
import sys
from tqdm.notebook import tqdm
import sqlite3


con = sqlite3.connect('./transaction_data.db')
cur = con.cursor()

def describe_table():
    cur.execute("select sql from sqlite_master where type = 'table' and name = 'transaction_data';")
    print(cur.fetchall()[0][0])

def get_first_record():
    cur.execute("select * from transaction_data LIMIT 1;")
    print(cur.fetchall()[0])

# (1, 1, 'PAYMENT', '9839.640', 'C1231006815', '170136.000', '160296.360', 'M1979787155', '0.000', '9839.640')

def export_csv(filename='./transactions.csv'):
    f = open(filename,'w')
    f.write('timestamp,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest\n')
    cur.execute("select * from transaction_data;")
    data = cur.fetchall()
    for line in tqdm(data):
        f.write(str(line[1])+','+line[2]+','+format(float(line[3]),'.2f')+','+line[4]+','+format(float(line[5]),'.2f')+','+format(float(line[6]),'.2f')+','+line[7]+','+format(float(line[8]),'.2f')+','+format(float(line[9]),'.2f')+'\n')

describe_table()
get_first_record()


CREATE TABLE transaction_data (
        id integer PRIMARY KEY,
        timestamp integer NOT NULL,
        type text NOT NULL,
        amount text NOT NULL,
        nameOrig text NOT NULL,
        oldbalanceOrig text NOT NULL,
        newbalanceOrig text NOT NULL,
        nameDest text NOT NULL,
        oldbalanceDest text NOT NULL,
        newbalanceDest text NOT NULL
    )
(1, 1, 'TRANSFER', '0.01', 'C1231006815', '170136.0', '170135.990', 'C52983754', '0.010', '0.020')


In [17]:
import pandas as pd
import numpy as np

def string_amount_to_int(i):
    if "E" in i:
        return float(i)*1000
    else:
        major, decimals = i.split(".")
#         assert len(decimals) <= 3, "Found a number with more than 3 decimal points:"+str(i)
        decimals = decimals[:3].ljust(3, "0")
        return int(major+decimals)

def read_to_pandas():
    cur.execute("select * from transaction_data;")
#     cur.execute("select * from transaction_data LIMIT 10000;")
    columns = 'timestamp,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest'.split(",")
    columns.insert(0, "tx_index")
    df = pd.DataFrame(data=cur.fetchall(), columns=columns)
    for col in ["tx_index", "timestamp", "amount", "oldbalanceOrig", "newbalanceOrig", "oldbalanceDest", "newbalanceDest"]:
        if col in ["amount", "oldbalanceOrig", "newbalanceOrig", "oldbalanceDest", "newbalanceDest"]:
        #Converting to an integer by factor of 1000!
            df[col] = df[col].apply(string_amount_to_int)
        else:
            df[col] = pd.to_numeric(df[col])
    return df

df = read_to_pandas()
df.set_index("tx_index", inplace=True, drop=False)
df.to_csv("df_transactions.csv")
df.dtypes

tx_index            int64
timestamp           int64
type               object
amount              int64
nameOrig           object
oldbalanceOrig    float64
newbalanceOrig      int64
nameDest           object
oldbalanceDest      int64
newbalanceDest      int64
dtype: object

In [18]:
!ls -all -h

total 1,4G
drwxr-xr-x  4 jupyter-maxim jupyter-maxim 4,0K feb 18 15:50 .
drwxr-x--- 22 jupyter-maxim jupyter-maxim 4,0K feb 18 15:50 ..
-rw-r--r--  1 jupyter-maxim jupyter-maxim 701M feb 18 15:50 df_transactions.csv
drwxr-xr-x  3 jupyter-maxim jupyter-maxim 4,0K feb 17 16:21 graphs
drwxr-xr-x  2 jupyter-maxim jupyter-maxim 4,0K feb 18 12:05 .ipynb_checkpoints
-rw-r--r--  1 jupyter-maxim jupyter-maxim 8,9K feb 18 15:49 office_space_hacker_analysis.ipynb
-rw-r--r--  1 jupyter-maxim jupyter-maxim 265K feb 18 15:50 SEN163A_Assignment1.ipynb
-rw-r--r--  1 jupyter-maxim jupyter-maxim 689M feb 18 11:46 transaction_data.db
-rw-r--r--  1 jupyter-maxim jupyter-maxim 7,4M feb 17 15:14 transactions_test_100k.csv
-rw-r--r--  1 jupyter-maxim jupyter-maxim 735K feb 17 15:14 transactions_test.csv
